<a href="https://colab.research.google.com/github/bonillahermes/Deep_Learning_Projects/blob/main/RegresionConySinANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hermes Yate Bonilla
**Data Scientist**
---

**Contact:**
- **Email:** [bonillahermes@gmail.com](mailto:bonillahermes@gmail.com)
- **LinkedIn:** [linkedin.com/in/bonillahermes](https://www.linkedin.com/in/bonillahermes/)
- **GitHub:** [github.com/bonillahermes](https://github.com/bonillahermes)
- **Webpage:** [bonillahermes.com](https://bonillahermes.com/)
---

# Modelo de Regresión con y Sin ANN

# Apartado A

## Descripción del Conjunto de Datos

El conjunto de datos utilizado en este análisis proviene de un archivo público sobre costos de seguros. Incluye información demográfica, hábitos y datos geográficos de los asegurados, junto con los cargos de seguro incurridos. Las variables específicas son:

- `Edad`: Edad del asegurado.
- `Sexo`: Género del asegurado (masculino o femenino).
- `BMI`: Índice de masa corporal, que proporciona una comprensión del cuerpo, pesos que son relativamente altos o bajos en relación con la altura, índice objetivo de peso corporal (kg/m^2).
- `Hijos`: Número de hijos/dependientes cubiertos por el seguro de salud.
- `Fumador`: Indica si el asegurado fuma o no.
- `Región`: El área residencial del asegurado en los EE. UU., dividida en cuatro regiones geográficas: noreste, sureste, suroeste, noroeste.
- `Cargos`: Costos médicos individuales facturados por el seguro de salud.

## Problema de Regresión

El objetivo principal de este análisis es predecir los `Cargos` de seguro de salud de un individuo, que es una variable continua, basándonos en las características demográficas y de comportamiento del asegurado. Este es un problema típico de regresión, donde se intenta estimar una variable de salida continua a partir de múltiples variables de entrada.

### Metodología

Para resolver este problema de regresión, se implementarán y compararán dos enfoques principales:

1. **Regresión con Modelos Tradicionales**: Se utilizarán métodos de regresión no basados en redes neuronales como regresión lineal o regresión forestal aleatoria para establecer una línea base de predicción.

2. **Redes Neuronales con TensorFlow**: Se empleará una red neuronal artificial utilizando TensorFlow, una biblioteca de aprendizaje profundo. Esta red será configurada y entrenada para predecir los cargos del seguro a partir de las variables de entrada. Se explorarán distintas arquitecturas de red para determinar la configuración más efectiva.

El rendimiento de los modelos se evaluará y comparará utilizando métricas adecuadas, como el Error Cuadrático Medio (MSE), para determinar la precisión de las predicciones en los datos de prueba. La elección del modelo final se basará tanto en la precisión de la predicción como en la comprensión de las características subyacentes que más influyen en los cargos del seguro.


In [ ]:
# Importaciones de bibliotecas estándar
import numpy as np
import matplotlib.pyplot as plt
import pathlib

# Importaciones de bibliotecas de terceros
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.regularizers import l1, l2
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

# Importaciones de scikit-learn para preprocesamiento, modelado y evaluación
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.pipeline import make_pipeline

# Imprimir la versión de TensorFlow
print(tf.__version__)


In [ ]:
# Cargar el conjunto de datos
url = "https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv"
df = pd.read_csv(url)

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
# Convertir las variables 'sex' y 'smoker' a valores binarios (0 y 1)
df['sex'] = df['sex'].apply(lambda x: 1 if x == 'male' else 0)
df['smoker'] = df['smoker'].apply(lambda x: 1 if x == 'yes' else 0)

# Convertir la variable 'region' a formato one-hot
df = pd.get_dummies(df, columns=['region'])

# Mostrar las primeras filas del DataFrame para verificar los cambios
print(df.head())

In [ ]:
train_df = df.sample(frac=0.8,random_state=0)
test_df = df.drop(train_df.index)

In [ ]:
train_stats = train_df.describe()
train_stats.pop("charges")
train_df.columns


In [ ]:
train_stats = train_stats.transpose()
train_stats

In [ ]:
sns.pairplot(train_df[["age", "bmi", "children", "charges"]], diag_kind="kde")

## Modelo de Regresión con Redes Neuronales

In [ ]:
train_labels = train_df.pop('charges')
test_labels = test_df.pop('charges')

def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_df)
normed_test_data = norm(test_df)

In [ ]:
def build_improved_model(input_shape):
    model = Sequential()
    # Añadir normalización por lotes
    model.add(BatchNormalization(input_shape=[input_shape]))
    # Añadir capas densas con regularización L2 y dropout para reducir el sobreajuste
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
    model.add(Dropout(0.5))
    model.add(Dense(1))

    # Utilizar Adam como optimizador con una tasa de aprendizaje reducida
    optimizer = Adam(learning_rate=0.0001)

    model.compile(loss='mse',
                  optimizer=optimizer,
                  metrics=['mae', 'mse'])
    return model

# Construir el modelo mejorado
improved_model = build_improved_model(input_shape=len(train_df.keys()))


# Implementar parada temprana
early_stop = EarlyStopping(monitor='val_loss', patience=10)

EPOCHS = 1000

# Entrenar el modelo con la parada temprana
history = improved_model.fit(normed_train_data, train_labels,
                             epochs=EPOCHS, validation_split=0.2, verbose=0,
                             callbacks=[early_stop])



In [ ]:
improved_model.summary()

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error [charges]')
    # Comprobamos si los datos están presentes antes de graficar
    if 'mae' in hist and 'val_mae' in hist:
        plt.plot(hist['epoch'], hist['mae'], label='Train Error')
        plt.plot(hist['epoch'], hist['val_mae'], label='Val Error')
    plt.legend()

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error [$charges^2$]')
    # Comprobamos si los datos están presentes antes de graficar
    if 'mse' in hist and 'val_mse' in hist:
        plt.plot(hist['epoch'], hist['mse'], label='Train Error')
        plt.plot(hist['epoch'], hist['val_mse'], label='Val Error')
    plt.legend()
    plt.show()

plot_history(history)


In [ ]:
loss, mae, mse = improved_model.evaluate(normed_test_data, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} charges".format(mae))

In [ ]:
loss, mae, mse = improved_model.evaluate(normed_train_data, train_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} charges".format(mae))

In [ ]:
test_predictions = improved_model.predict(normed_test_data).flatten()

# test_labels contiene los valores reales
# test_predictions contiene los valores predichos por el modelo

# Gráfico de dispersión de las etiquetas verdaderas vs predicciones
plt.scatter(test_labels, test_predictions)

# Etiquetas de los ejes
plt.xlabel('True Values [charges]')
plt.ylabel('Predictions [charges]')

# Establecer un aspecto de eje igual y cuadrado para la gráfica
plt.axis('equal')
plt.axis('square')

# Establecer los límites de los ejes para tener el mismo rango
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])

# Dibujar una línea y=x para referencia
plt.plot([-100, 100], [-100, 100])

# Ajustar un modelo lineal a los datos para obtener la línea de tendencia
z = np.polyfit(test_labels, test_predictions, 1)
p = np.poly1d(z)

# Dibujar la línea de tendencia
plt.plot(test_labels, p(test_labels), "r--")

# Mostrar la gráfica
plt.show()


In [ ]:
error = test_predictions - test_labels
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [charges]")
_ = plt.ylabel("Count")

# Modelo de regresión sin Usar Redes Neuronales

In [ ]:
X = df.drop('charges', axis=1)  # Elimina la columna 'charges' y usa el resto como características
y = df['charges']  # Usa la columna 'charges' como la variable objetivo

# Crear características polinómicas
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X)

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo de regresión Ridge
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de entrenamiento y prueba
train_predictions = ridge_model.predict(X_train)
test_predictions = ridge_model.predict(X_test)

# Evaluar el modelo usando validación cruzada
scores = cross_val_score(ridge_model, X_poly, y, cv=5, scoring='neg_mean_squared_error')

# Calcular el error para el conjunto de entrenamiento y prueba
train_mae = mean_absolute_error(y_train, train_predictions)
train_mse = mean_squared_error(y_train, train_predictions)
test_mae = mean_absolute_error(y_test, test_predictions)
test_mse = mean_squared_error(y_test, test_predictions)

# Imprimir errores y puntuación de validación cruzada
print(f'Train MAE: {train_mae}')
print(f'Train MSE: {train_mse}')
print(f'Test MAE: {test_mae}')
print(f'Test MSE: {test_mse}')
print(f'Cross-Validation MSE: {-scores.mean()}')


In [ ]:
# Calcular los residuos (diferencia entre la predicción y el valor real)
train_residuals = y_train - train_predictions
test_residuals = y_test - test_predictions

# Gráfico de dispersión para los residuos del conjunto de entrenamiento
plt.scatter(train_predictions, train_residuals, label='Train Data', alpha=0.5)
plt.hlines(y=0, xmin=train_predictions.min(), xmax=train_predictions.max(), colors='r', linestyles='--')
plt.title('Residuals for Training Data')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.legend()
plt.show()

# Gráfico de dispersión para los residuos del conjunto de prueba
plt.scatter(test_predictions, test_residuals, label='Test Data', alpha=0.5)
plt.hlines(y=0, xmin=test_predictions.min(), xmax=test_predictions.max(), colors='r', linestyles='--')
plt.title('Residuals for Test Data')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.legend()
plt.show()

In [ ]:
# Realizar predicciones en el conjunto de prueba
test_predictions = ridge_model.predict(X_test).flatten()

# Gráfico de dispersión de las etiquetas verdaderas vs predicciones
plt.scatter(y_test, test_predictions)

# Etiquetas de los ejes
plt.xlabel('True Values [charges]')
plt.ylabel('Predictions [charges]')

# Establecer un aspecto de eje igual y cuadrado para la gráfica
plt.axis('equal')
plt.axis('square')

# Establecer los límites de los ejes para tener el mismo rango
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])

# Dibujar una línea y=x para referencia
plt.plot([-100, 100], [-100, 100])

# Ajustar un modelo lineal a los datos para obtener la línea de tendencia
z = np.polyfit(y_test, test_predictions, 1)
p = np.poly1d(z)

# Dibujar la línea de tendencia
plt.plot(y_test, p(y_test), "r--")

# Mostrar la gráfica
plt.show()


In [ ]:
# Calcular el error de predicción
error = test_predictions- y_test

# Crear el histograma de los errores de predicción
plt.hist(error, bins=25)
plt.xlabel("Prediction Error [charges]")
plt.ylabel("Count")
plt.show()

In [ ]:
# errors_nn es un array de los errores de predicción de la red neuronal
# errors_lr es un array de los errores de predicción de la regresión lineal
errors_nn = improved_model.predict(normed_test_data).flatten() - test_labels
errors_lr = ridge_model.predict(X_test).flatten() - y_test

# Convertir a valores absolutos
errors_nn_abs = np.abs(errors_nn)
errors_lr_abs = np.abs(errors_lr)

# Crear una estructura de datos adecuada para el boxplot
data_to_plot = [errors_nn_abs, errors_lr_abs]
labels = ['Neural Network', 'Linear Regression']

# Crear el boxplot
plt.figure(figsize=(10, 8))
sns.boxplot(data=data_to_plot)
plt.xticks(range(len(labels)), labels)
plt.ylabel('Absolute Prediction Error [charges]')
plt.title('Comparison of Prediction Errors between Models')
plt.show()


# Apartado B

# Descripción del Conjunto de Datos y Problema de Clasificación

## Contexto del Conjunto de Datos

El conjunto de datos utilizado se enfoca en los clientes de un servicio de tarjetas de crédito de un banco. Contiene información detallada de 10,000 clientes, incluyendo características como edad, salario, estado civil, límite de crédito y categoría de la tarjeta de crédito. Una característica destacada es el nivel educativo de los clientes, el cual se presenta en distintas categorías.

## Características del Conjunto de Datos

- **Número de Observaciones**: 10,000
- **Número de Características**: Aproximadamente 18
- **Variables Clave**:
  - **Nivel Educativo**: Categoría principal de interés.
  - **Datos Demográficos**: Edad, estado civil.
  - **Datos Financieros**: Salario, límite de crédito.
  - **Uso de la Tarjeta de Crédito**: Categoría de la tarjeta, etc.

## Problema de Clasificación

### Objetivo

El objetivo principal es clasificar a los clientes en categorías de nivel educativo. El nivel educativo es una variable categórica que podría tener clases como "Secundaria", "Universitaria", "Postgrado", etc.

### Importancia

Comprender la distribución del nivel educativo entre los clientes puede ayudar al banco a ofrecer productos más personalizados y mejorar su estrategia de marketing.

### Desafíos

- **Equilibrio de Clases**: Si algunas categorías educativas son mucho menos comunes que otras, esto podría presentar un desafío en términos de balanceo de clases.
- **Relaciones No Lineales**: Las relaciones entre el nivel educativo y otras características podrían no ser lineales o podrían ser influenciadas por factores ocultos.

### Métodos Propuestos

- **Análisis Exploratorio de Datos**: Para entender la distribución de las distintas características, especialmente el nivel educativo.
- **Modelos de Clasificación Multinomial**: Dado que el objetivo es clasificar cada observación en una de varias categorías de nivel educativo, los modelos como Regresión Logística Multinomial, Árboles de Decisión o Random Forest pueden ser adecuados.
- **Validación Cruzada**: Para evaluar la eficacia de los modelos y asegurar que funcionen bien en datos no vistos.
- **Técnicas de Balanceo de Clases**: En caso de desequilibrio en las clases, se podrían usar técnicas como SMOTE para equilibrar las categorías del nivel educativo.

El enfoque en el nivel educativo y su relación con otras variables proporciona una oportunidad valiosa para entender mejor la base de clientes del banco y mejorar la toma de decisiones basada en datos.


In [ ]:
# Importaciones de bibliotecas estándar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Importaciones de sklearn para preprocesamiento y evaluación de modelos
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Importaciones de imblearn para manejo de desbalance en los datos
from imblearn.over_sampling import SMOTE

# Importaciones de keras para construcción y entrenamiento de modelos de deep learning
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam


In [ ]:
# Cargar el conjunto de datos
url = "https://raw.githubusercontent.com/bonillahermes/Datasets/main/BankChurners.csv"
df = pd.read_csv(url)

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
# Filtrando el conjunto de datos para omitir las categorías 'Unknown' en 'Marital_Status' y 'Education_Level'
df= df[(df['Marital_Status'] != 'Unknown') & (df['Education_Level'] != 'Unknown')]

# Mostrando las primeras filas del conjunto de datos filtrado para verificar
df.head()

In [ ]:
# Eliminando las columnas especificadas del dataframe df
columns_to_drop = ['CLIENTNUM',
                   'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
                   'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2']

df = df.drop(columns=columns_to_drop)

# Mostrando las primeras filas del dataframe modificado para verificar
df.head()

## Modelo de Clasificación No Basado en Redes Neuronales

In [ ]:
# Codificar las variables categóricas
le = LabelEncoder()
for column in df.columns:
    if df[column].dtype == object:
        df[column] = le.fit_transform(df[column])

# Dividiendo el conjunto de datos
X = df.drop('Education_Level', axis=1)  # Características
y = df['Education_Level']  # Variable objetivo

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Aplicar SMOTE solo al conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)



In [ ]:
# Crear el modelo de Random Forest con los mejores hiperparámetros encontrados
best_rf = RandomForestClassifier(
    n_estimators=300,
    min_samples_split=2,
    min_samples_leaf=1,
    max_depth=None,
    bootstrap=False,
    random_state=42
)

# Entrenar el modelo con los datos balanceados
best_rf.fit(X_train_smote, y_train_smote)

# Realizar predicciones en el conjunto de prueba
y_pred_rf = best_rf.predict(X_test)

# Imprimir el informe de clasificación
print(classification_report(y_test, y_pred_rf))



In [ ]:
# Calculando las métricas para el modelo de Random Forest
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf, average='weighted')
recall_rf = recall_score(y_test, y_pred_rf, average='weighted')
f1_score_rf = f1_score(y_test, y_pred_rf, average='weighted')

print(accuracy_rf)
print(precision_rf)
print(recall_rf)
print(f1_score_rf)

# Creando listas de métricas (solo Random Forest en este caso)
metrics_rf = [accuracy_rf, precision_rf, recall_rf, f1_score_rf]
labels = ['Accuracy', 'Precision', 'Recall', 'F1 Score']

# Graficando las métricas
x = range(len(labels))
width = 0.35

fig, ax = plt.subplots()
rects = ax.bar(x, metrics_rf, width, label='Random Forest')

# Añadiendo títulos y etiquetas
ax.set_ylabel('Scores')
ax.set_title('Métricas del Modelo de Random Forest')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

plt.show()


# Modelo de Clasificación de Red Neuronal Usandos Keras

In [ ]:
# Normalización de las características
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Dividiendo el conjunto de datos
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.3, random_state=42)

# Construyendo el modelo de red neuronal mejorado
model_nn = Sequential()
model_nn.add(Dense(128, input_dim=X_train.shape[1]))
model_nn.add(BatchNormalization())
model_nn.add(Activation('relu'))
model_nn.add(Dropout(0.5))
model_nn.add(Dense(64))
model_nn.add(BatchNormalization())
model_nn.add(Activation('relu'))
model_nn.add(Dropout(0.5))
model_nn.add(Dense(y.nunique(), activation='softmax'))  # Número de clases en y

# Compilando el modelo con un optimizador Adam ajustado
optimizer = Adam(learning_rate=0.001)
model_nn.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# Entrenando el modelo con callbacks
model_nn.fit(X_train, y_train, epochs=1000, batch_size=32, validation_split=0.2, callbacks=[early_stopping, model_checkpoint])

# Cargar el mejor modelo guardado y evaluar en el conjunto de prueba
model_nn.load_weights('best_model.h5')
_, accuracy = model_nn.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy*100:.2f}%')



In [ ]:
# Hacer predicciones
y_pred_nn = model_nn.predict(X_test)
y_pred_nn = y_pred_nn.argmax(axis=1)  # Convertir predicciones de probabilidades a etiquetas

In [ ]:
# Calcular métricas
accuracy_nn = accuracy_score(y_test, y_pred_nn)
precision_nn = precision_score(y_test, y_pred_nn, average='weighted')
recall_nn = recall_score(y_test, y_pred_nn, average='weighted')
f1_score_nn = f1_score(y_test, y_pred_nn, average='weighted')

In [ ]:
# Creando listas de métricas para el modelo de red neuronal
metrics_nn = [accuracy_nn, precision_nn, recall_nn, f1_score_nn]
labels = ['Accuracy', 'Precision', 'Recall', 'F1 Score']

# Graficando las métricas del modelo de red neuronal
x = range(len(labels))
width = 0.35

fig, ax = plt.subplots()
rects = ax.bar(x, metrics_nn, width, label='Red Neuronal')

# Añadiendo títulos y etiquetas
ax.set_ylabel('Scores')
ax.set_title('Métricas del Modelo de Red Neuronal')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

plt.show()
